In [2]:
from keras.layers import Conv2D, SeparableConv2D, Input
from keras.models import Model

In [3]:
input_image = Input((224, 224, 3))
feature_maps = Conv2D(filters=32, kernel_size=(3,3))(input_image)
feature_maps2 = Conv2D(filters=64, kernel_size=(3,3))(feature_maps)
model = Model(inputs=input_image, outputs=feature_maps2)

In [4]:
model.summary()

Model: "functional_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_1 (InputLayer)         [(None, 224, 224, 3)]     0         
_________________________________________________________________
conv2d (Conv2D)              (None, 222, 222, 32)      896       
_________________________________________________________________
conv2d_1 (Conv2D)            (None, 220, 220, 64)      18496     
Total params: 19,392
Trainable params: 19,392
Non-trainable params: 0
_________________________________________________________________


# 同樣模型，改用序列式建立:

In [7]:
#CNN相關套件模組
import keras
from keras import layers
from keras import models
from keras.models import Sequential
from keras.layers import Conv2D, Activation, MaxPooling2D, Flatten, Dense

model = models.Sequential()
model.add(Conv2D(32,(3,3),input_shape=(224,224,3)))
model.add(Conv2D(64,(3,3)))

model.summary()

Model: "sequential_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_4 (Conv2D)            (None, 222, 222, 32)      896       
_________________________________________________________________
conv2d_5 (Conv2D)            (None, 220, 220, 64)      18496     
Total params: 19,392
Trainable params: 19,392
Non-trainable params: 0
_________________________________________________________________


## 可以看到經過兩次 Conv2D，如果沒有設定 padding="SAME"，圖就會越來越小，同時特徵圖的 channel 數與 filters 的數量一致

# SeparableConv2D:

全名稱做 Depthwise Separable Convolution，與常用的 Conv2D 效果類似，但是參數量可以大幅減少，減輕對硬體的需求

對影像做兩次卷積，第一次稱為 DetphWise Conv，對影像的三個通道獨立做卷積，得到三張特徵圖；第二次稱為 PointWise Conv，使用 1x1 的 filter 尺寸做卷積。兩次卷積結合起來可以跟常用的卷積達到接近的效果，但參數量卻遠少於常見的卷積


In [9]:
input_image = Input((224, 224, 3))
feature_maps = SeparableConv2D(filters=32, kernel_size=(3,3))(input_image)
feature_maps2 = SeparableConv2D(filters=64, kernel_size=(3,3))(feature_maps)
model = Model(inputs=input_image, outputs=feature_maps2)

model.summary()

Model: "functional_5"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_3 (InputLayer)         [(None, 224, 224, 3)]     0         
_________________________________________________________________
separable_conv2d_2 (Separabl (None, 222, 222, 32)      155       
_________________________________________________________________
separable_conv2d_3 (Separabl (None, 220, 220, 64)      2400      
Total params: 2,555
Trainable params: 2,555
Non-trainable params: 0
_________________________________________________________________


# 也用序列式建立看看:

In [11]:
#CNN相關套件模組
import keras
from keras import layers
from keras import models
from keras.models import Sequential
from keras.layers import Conv2D, Activation, MaxPooling2D, Flatten, Dense

model = models.Sequential()
model.add(SeparableConv2D(32,(3,3),input_shape=(224,224,3)))
model.add(SeparableConv2D(64,(3,3)))

model.summary()

Model: "sequential_2"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
separable_conv2d_4 (Separabl (None, 222, 222, 32)      155       
_________________________________________________________________
separable_conv2d_5 (Separabl (None, 220, 220, 64)      2400      
Total params: 2,555
Trainable params: 2,555
Non-trainable params: 0
_________________________________________________________________


## 可以看到使用 Seperable Conv2D，即使模型設置都一模一樣，但是參數量明顯減少非常多！

## 作業

請閱讀 Keras 官方範例 [mnist_cnn.py](https://github.com/keras-team/keras/blob/master/examples/mnist_cnn.py)  

並回答下列問題。僅有 70 行程式碼，請確保每一行的程式碼你都能夠理解目的

1. 是否有對資料做標準化 (normalization)? 如果有，在哪幾行?
2. 使用的優化器 Optimizer 為何?
3. 模型總共疊了幾層卷積層?
4. 模型的參數量是多少?


In [14]:
'''Trains a simple convnet on the MNIST dataset.
Gets to 98.97% test accuracy after 12 epochs
'''

from __future__ import print_function
import keras
from keras.datasets import mnist
from keras.models import Sequential
from keras.layers import Dense, Dropout, Flatten
from keras.layers import Conv2D, MaxPooling2D
from keras import backend as K

# 基本設定:
batch_size = 128
num_classes = 10
epochs = 12

# input image dimensions
img_rows, img_cols = 28, 28

# the data, shuffled and split between train and test sets
(x_train, y_train), (x_test, y_test) = mnist.load_data()

print(x_train.shape)
print(x_test.shape)
print(y_train.shape)
print(y_test.shape)


(60000, 28, 28)
(10000, 28, 28)
(60000,)
(10000,)


In [16]:
#改為適合的資料維度:
if K.image_data_format() == 'channels_first':
    x_train = x_train.reshape(x_train.shape[0], 1, img_rows, img_cols)
    x_test = x_test.reshape(x_test.shape[0], 1, img_rows, img_cols)
    input_shape = (1, img_rows, img_cols)
else:
    x_train = x_train.reshape(x_train.shape[0], img_rows, img_cols, 1)
    x_test = x_test.reshape(x_test.shape[0], img_rows, img_cols, 1)
    input_shape = (img_rows, img_cols, 1)

print(x_train.shape)
print(x_test.shape)
#print(y_train.shape)
#print(y_test.shape)

(60000, 28, 28, 1)
(10000, 28, 28, 1)


In [17]:
#資料標準化
x_train = x_train.astype('float32')
x_test = x_test.astype('float32')
x_train /= 255
x_test /= 255

print('x_train shape:', x_train.shape)
print(x_train.shape[0], 'train samples')
print(x_test.shape[0], 'test samples')

x_train shape: (60000, 28, 28, 1)
60000 train samples
10000 test samples


In [18]:
# convert class vectors to binary class matrices
y_train = keras.utils.to_categorical(y_train, num_classes)
y_test = keras.utils.to_categorical(y_test, num_classes)

In [19]:
#建立序列式模型:
model = Sequential()

model.add(Conv2D(32, kernel_size=(3, 3),
                 activation='relu',
                 input_shape=input_shape)) #輸入層 & 卷積層1

model.add(Conv2D(64, (3, 3), activation='relu')) #卷積層2

model.add(MaxPooling2D(pool_size=(2, 2))) #池化層1

model.add(Dropout(0.25)) #隨機丟棄

model.add(Flatten()) #維度拉平

model.add(Dense(128, activation='relu')) #全連接層

model.add(Dropout(0.5)) #隨機丟棄

model.add(Dense(num_classes, activation='softmax')) #全連接層 & 輸出層

model.compile(loss=keras.losses.categorical_crossentropy,
              optimizer=keras.optimizers.Adadelta(),
              metrics=['accuracy'])

model.fit(x_train, y_train,
          batch_size=batch_size,
          epochs=epochs,
          verbose=1,
          validation_data=(x_test, y_test))

model.summary()

Epoch 1/12
469/469 [==============================] - 140s 299ms/step - loss: 2.2849 - accuracy: 0.1382 - val_loss: 2.2457 - val_accuracy: 0.2931
Epoch 2/12
469/469 [==============================] - 140s 300ms/step - loss: 2.2246 - accuracy: 0.2388 - val_loss: 2.1732 - val_accuracy: 0.4391
Epoch 3/12
469/469 [==============================] - 145s 308ms/step - loss: 2.1501 - accuracy: 0.3338 - val_loss: 2.0785 - val_accuracy: 0.5605
Epoch 4/12
469/469 [==============================] - 140s 299ms/step - loss: 2.0519 - accuracy: 0.4121 - val_loss: 1.9531 - val_accuracy: 0.6352
Epoch 5/12
469/469 [==============================] - 141s 300ms/step - loss: 1.9264 - accuracy: 0.4798 - val_loss: 1.7928 - val_accuracy: 0.6886
Epoch 6/12
469/469 [==============================] - 141s 300ms/step - loss: 1.7749 - accuracy: 0.5387 - val_loss: 1.6025 - val_accuracy: 0.7233
Epoch 7/12
469/469 [==============================] - 141s 300ms/step - loss: 1.6078 - accuracy: 0.5784 - val_loss: 1.4008 -

In [ ]:
作業１：是否有對資料做標準化 (normalization)？ 如果有，在哪幾行？

Ans:有，除以255的那段

作業２：使用的優化器 Optimizer 為何？ 

Ans:使用Adadelta

作業３：模型總共疊了幾層卷積層？ 

Ans:2層

作業４：模型的參數量是多少？

Ans:一共1,199,882個參數

# 作業１：是否有對資料做標準化 (normalization)？ 如果有，在哪幾行？

  Ans:有，除以255的那段

# 作業２：使用的優化器 Optimizer 為何？ 

  Ans:使用Adadelta

# 作業３：模型總共疊了幾層卷積層？ 

  Ans:2層

# 作業４：模型的參數量是多少？

  Ans:一共1,199,882個參數

In [20]:
#模型準確率評估:
score = model.evaluate(x_test, y_test, verbose=0)

print('Test loss:', score[0])
print('Test accuracy:', score[1])

Test loss: 0.7566702365875244
Test accuracy: 0.8253999948501587
